In [1]:
import os
# Per avere una traccia più precisa dell'errore
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import math
import numpy as np

from scipy import signal
from scipy.fft import fft, fftshift

import torch
import torch.nn as nn
from torch.nn import functional as F

import cv2
import pickle
from PIL import Image
import matplotlib.pyplot as plt

import IPython
from tqdm import tqdm

In [2]:
!pip install transformers
!pip install librosa

from transformers import ViTModel, ViTConfig, AdamW

import librosa
import librosa.display as display

2023-06-03 15:55:11.173055: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [3]:
CUDA_VISIBLE_DEVICES=7
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
PYTORCH_CUDA_ALLOC_CONF=1.1

In [5]:
class ResidualBlock(nn.Module):

    def __init__(self, device='cpu'):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
                        nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=(3,3), padding=(2,1), stride=(2,1)), 
                        nn.GELU(), 
                        nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=(3,3), padding=1, stride=1)
        ).to(device) 
        
        self.ext_block = nn.Sequential(
                        nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=(3,3), padding=(2,1), stride=(2,1))
        ).to(device) 
        
    def forward(self, inputs):
        extended_input = self.ext_block(inputs)
        convolved_input = self.block(inputs)
        return convolved_input + extended_input

In [6]:
class ConvResidualBlock(nn.Module):

    def __init__(self, device='cpu'):
        super(ConvResidualBlock, self).__init__()
        self.block = nn.Sequential(
                        nn.Conv2d(1, 1, 3, stride=1, padding="same"), 
                        nn.Conv2d(1, 1, 3, stride=1, padding="same")
        ).to(device) 
        
        self.conv_1d = nn.Conv2d(1, 1, 3, stride=1, padding="same").to(device)
        
    def forward(self, inputs):
        convolved_input = self.block(inputs)
        skip_con = self.conv_1d(inputs)
        return convolved_input + skip_con

In [7]:
class GenerativeNetwork(nn.Module):
    """
    Input Shape: (b, 1, 800, 800)
    Output ViT: (b, num_pathces, hidden_size)
    After Reshape: (b, 1, x, x) dove x è una dimensione generica che puoi decidere
    Ouput Shape: (b, 1, 1025, 800)
    """
    
    def __init__(self, device='cpu'):
        super(GenerativeNetwork, self).__init__()
        self.device = device
        self.hidden_size = 1312
        self.patch_size = 32
        configuration = ViTConfig(num_attention_heads=8, 
                                  num_hidden_layers=8,
                                  hidden_size=self.hidden_size,
                                  patch_size=self.patch_size,
                                  num_channels=1, 
                                  image_size=800)
        self.vit = ViTModel(configuration).to(self.device)
        self.refine_model = nn.Sequential(*[ConvResidualBlock() for _ in range(3)]).to(device)
        
    
    def patch_to_img(self, x):
        row_patch_size = 41
        col_patch_size = 32
        B, NumPatches, HiddenSize = x.shape
        x = x.reshape(B, NumPatches, 1, HiddenSize)
        x = x.reshape(B, NumPatches, 1, row_patch_size, col_patch_size)
        x = x.permute(0, 1, 3, 4, 2)
        x = x.reshape(B, int(math.sqrt(NumPatches)), int(math.sqrt(NumPatches)), row_patch_size, col_patch_size, 1)
        x = x.permute(0,1,3,2,4,5)
        new_h = x.shape[1] * x.shape[2]
        new_w = x.shape[3] * x.shape[4]
        x = x.reshape(B, new_h, new_w, 1) #ultima posizione = num_channels che è sempre 1
        x = x.swapaxes(3, 1)
        x = x.swapaxes(3, 2)
        return x
    
        
    def forward(self, inputs):
        if inputs.device == 'cpu':
            inputs = inputs.to(self.device)
        vit_res = self.vit(pixel_values=inputs)
        inputs = vit_res.last_hidden_state[:, 1:, :]
        #patch_size_after_vit = int(math.sqrt(inputs.shape[2]))
        inputs = self.patch_to_img(inputs) #, patch_size_after_vit)
        return self.refine_model(inputs)

In [8]:
class DiscriminativeNetwork(nn.Module): 
    
    def __init__(self, device='cpu'):
        super(DiscriminativeNetwork, self).__init__()
        self.device = device
        self.classifier = nn.Sequential(
                                        nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, stride=2),
                                        nn.LeakyReLU(0.2),
                                        nn.BatchNorm2d(2),
                                        nn.Conv2d(in_channels=2, out_channels=4, kernel_size=3, stride=2),
                                        nn.LeakyReLU(0.2),
                                        nn.BatchNorm2d(4),
                                        nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, stride=2),
                                        nn.LeakyReLU(0.2),
                                        nn.BatchNorm2d(8),
                                        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2),
                                        nn.LeakyReLU(0.2),
                                        nn.BatchNorm2d(16),
                                        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2),
                                        nn.LeakyReLU(0.2),
                                        nn.BatchNorm2d(32),
                                        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2), 
                                        nn.LeakyReLU(0.2),
                                        nn.BatchNorm2d(64),
                                        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2), 
                                        nn.LeakyReLU(0.2),
                                        nn.BatchNorm2d(128),
                                        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2), 
                                        nn.LeakyReLU(0.2),
                                        nn.BatchNorm2d(256),
                                        nn.Flatten(),
                                        nn.Dropout(0.3),
                                        nn.Linear(in_features=1536, out_features=128),
                                        nn.LeakyReLU(0.2),
                                        nn.Dropout(0.3),
                                        nn.Linear(in_features=128, out_features=1),
                                        nn.Sigmoid()
                                        
                                        
        ).to(self.device)


    def forward(self, inputs):
        if inputs.device == 'cpu':
            inputs = inputs.to(self.device)
        return self.classifier(inputs)

In [9]:
class LHB_Dataset(torch.utils.data.Dataset):

    def __init__(self, path, ext):
        self.path = path
        self.ext = ext
        self.len = len(os.listdir(self.path))
        self.items_in_dir = os.listdir(self.path)


    def __len__(self):
        return self.len

    
    def __getitem__(self, idx):
       
        name = self.path + '/' + self.items_in_dir[idx] #self.path + '/' + str(idx) + "." + self.ext

        with open(name, 'rb') as fd:
            song = pickle.load(fd)

        return song #[:1321967]

In [10]:
train_path = "/home/simona/Adele/UnzippedDataset/train"

train_ds = LHB_Dataset(train_path, 'mus')

print(train_ds[0].shape)
print(len(train_ds))

(1321967,)
2664


In [11]:
validation_path = '/home/simona/Adele/UnzippedDataset/validation'

validation_ds = LHB_Dataset(validation_path, 'mus')

print(validation_ds[0].shape)
print(len(validation_ds))

(1323119,)
572


In [12]:
#train
train_generator = torch.Generator(device='cpu')
train_generator.manual_seed(13)
trainloader = torch.utils.data.DataLoader(
                                            dataset=train_ds, 
                                            batch_size=1, 
                                            shuffle=True,
                                            generator=train_generator
                                        )

#validation
validation_generator = torch.Generator(device='cpu')
validation_generator.manual_seed(13)
validloader = torch.utils.data.DataLoader(
                                            dataset=validation_ds, 
                                            batch_size=1, 
                                            shuffle=False,
                                            generator=validation_generator
                                        )

In [13]:
# Models
generator = GenerativeNetwork(device).to(device)
discriminator = DiscriminativeNetwork(device).to(device)

# Optimizers
optimizer_gen = AdamW(generator.parameters(), lr=1e-4, weight_decay=1e-4) 
optimizer_dis = torch.optim.Adam(discriminator.parameters(), lr=1e-5) 

# Loss
loss_gen = nn.MSELoss()
loss_dis = nn.BCELoss()

/home/simona/adele_venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
def train_with_val(trainloader, validloader, generator, discriminator, optimizer_gen, optimizer_dis, loss_gen, loss_dis, name_to_save, epoches=1, start=0, beta=1.0, best_val=float('inf'), device='cpu'): 
    from tqdm import tqdm
    
    TRAIN_CHECKPOINTS_DIR_PATH = '.'
    filename = 'afterIsmis_test1203_100.txt'
    
    alpha = 1.5
    
    best_val_loss = best_val
    old_valid_discriminator_loss = float('inf')
    
    NUM_COLS = 800
    NUM_ROWS_LB = 800
    
    GRAD_ACC = 8

    # TrainSteps
    for epoch in tqdm(range(epoches), desc='Epoch '):
        generator.train()
        discriminator.train()
        num_samples_seen = 0
        total_gen_loss = 0
        total_dis_loss = 0
        
        # Iter on batches
        num_songs = 0
        for data_batch in trainloader: 
            print(f'Song n.{num_songs} on {len(trainloader)}')
            batch_lf = []
            batch_hf = []
            
            for data in data_batch:
                
                data = data.squeeze(dim=0) # one song 
                
                # Compute spectrograms
                train_stft = librosa.stft(np.asarray(data), n_fft=4096, win_length=4096, window=signal.windows.hamming(4096))
                train_spectrogram = torch.tensor(librosa.amplitude_to_db(abs(train_stft)))

                rows = train_spectrogram.shape[0]
                cols = train_spectrogram.shape[1]

                if cols % NUM_COLS > 0:
                    cols_to_add = NUM_COLS - cols % NUM_COLS
                    new_data = torch.zeros(size = (rows, cols + cols_to_add))
                    new_data[:, : cols] = train_spectrogram
                    new_data[:, cols : cols+cols_to_add] = train_spectrogram[:, -cols_to_add:]
                    train_spectrogram = new_data
                    cols = cols + cols_to_add
                    
                train_spectrogram = train_spectrogram.reshape(1, rows, cols).float()

                PTS = cols // NUM_COLS

                for i in range(PTS):
                    batch_lf.append(train_spectrogram[:, : 800, i*NUM_COLS : (i+1)*NUM_COLS])
                    batch_hf.append(train_spectrogram[:, 800 : 1825 , i*NUM_COLS : (i+1)*NUM_COLS])

                
            batch_lf = torch.stack(batch_lf).to(device)
            batch_hf = torch.stack(batch_hf).to(device)
            
            num_samples_seen += batch_lf.shape[0]
            
            shuffled_indexes = np.random.permutation(batch_lf.shape[0]) #shuffle
            batch_lf = batch_lf[shuffled_indexes]
            batch_hf = batch_hf[shuffled_indexes]
                        
            # Train the discriminator on the true/generated data
            generated_data = generator(batch_lf)
            combined_data = torch.cat((batch_hf.to(device), generated_data), dim=0)          
            labels = torch.cat((torch.ones(batch_hf.shape[0]), torch.zeros(generated_data.shape[0])), dim=0)
            
            shuffled_indexes = np.random.permutation(combined_data.shape[0]) #shuffle
            combined_data = combined_data[shuffled_indexes]
            labels = labels[shuffled_indexes].to(device)
                
            discriminator_out = discriminator(combined_data).reshape(-1)
            #print(f'DIS: {discriminator_out.shape}, LABELS: {labels.shape}')
            optimizer_dis.zero_grad()
            discriminator_loss = loss_dis(discriminator_out, labels)
            discriminator_loss.backward()
            optimizer_dis.step()           
            
            # Train the generator
            optimizer_gen.zero_grad()
            generator_out = generator(batch_lf)
            generator_loss = loss_gen(batch_hf, generator_out)
            
            discriminator_out_gen = discriminator(generator_out.detach()).reshape(-1)
            discriminator_loss_gen = loss_dis(discriminator_out_gen.to('cpu'), torch.ones(size=(discriminator_out_gen.shape[0],))) #bce
                        
            total_dis_loss = total_dis_loss + discriminator_loss_gen.detach()
            total_gen_loss = total_gen_loss + generator_loss.detach()

            loss = alpha*generator_loss + beta*discriminator_loss_gen 

            loss.backward()
            optimizer_gen.step()
            
            
            num_songs += 1
        
        #if num_songs % GRAD_ACC != 0:
        """
        optimizer_gen.step()
        optimizer_gen.zero_grad()
        optimizer_dis.step()
        optimizer_dis.zero_grad()
        """
        
        # End Trainloader Loop
        
        print('START VALIDATION')
        # Validation
        with torch.no_grad():
            generator.eval()
            discriminator.eval()
            total_val_gen = 0
            total_val_dis = 0
                        
            valid_samples_seen = 0
            
            for valid_data in validloader:
                valid_data = valid_data.squeeze()
                valid_stft = librosa.stft(np.asarray(valid_data), n_fft=4096, win_length=4096, window=signal.windows.hamming(4096))
                valid_spectrogram = torch.tensor(librosa.amplitude_to_db(abs(valid_stft)))

                valid_rows = valid_spectrogram.shape[0]
                valid_cols = valid_spectrogram.shape[1]

                if valid_cols % NUM_COLS > 0:
                    cols_to_add = NUM_COLS - valid_cols % NUM_COLS
                    new_data = torch.zeros(size = (valid_rows, valid_cols + cols_to_add))
                    new_data[:, : valid_cols] = valid_spectrogram
                    new_data[:, valid_cols : valid_cols+cols_to_add] = valid_spectrogram[:, -cols_to_add:]
                    valid_spectrogram = new_data
                    valid_cols = valid_cols + cols_to_add
                    
                valid_spectrogram = valid_spectrogram.reshape(1, 1, valid_rows, valid_cols)

                VALID_PTS = valid_cols // NUM_COLS
                
                valid_samples_seen = valid_samples_seen + VALID_PTS

                valid_lf = torch.zeros(size=(VALID_PTS, 1, 800, NUM_COLS))
                valid_hf = torch.zeros(size=(VALID_PTS, 1, 1025, NUM_COLS)).to(device)

                for j in range(VALID_PTS):
                    valid_lf[i, :, :, :] = valid_spectrogram[:, :, : 800, i*NUM_COLS : (i+1)*NUM_COLS]
                    valid_hf[i, :, :, :] = valid_spectrogram[:, :, 800 : 1825, i*NUM_COLS : (i+1)*NUM_COLS]

                valid_hf_generated = generator(valid_lf.to(device))
                
                valid_gen_loss = loss_gen(valid_hf.to('cpu'), valid_hf_generated.to('cpu'))
                                
                valid_dis_out = discriminator(valid_hf_generated.detach()).reshape(-1)
                valid_dis_loss = loss_dis(valid_dis_out.to('cpu'), torch.ones(size=(valid_dis_out.shape[0],)))
                
                total_val_gen = total_val_gen + valid_gen_loss
                total_val_dis = total_val_dis + valid_dis_loss
                
            mean_val_loss_gen = total_val_gen / valid_samples_seen
            mean_val_loss_dis = total_val_dis / valid_samples_seen
            
            mean_val_loss = alpha*mean_val_loss_gen + beta*mean_val_loss_dis
            
            if mean_val_loss < best_val_loss:
                best_val_loss = mean_val_loss
                print('UPDATE TO ', best_val_loss)
                gen_name = os.path.join(TRAIN_CHECKPOINTS_DIR_PATH, 'GEN_BestVal_'+name_to_save)
                torch.save({
                    'epoch': epoch+start,
                    'model_state_dict': generator.state_dict(),
                    'optimizer_state_dict': optimizer_gen.state_dict(),
                    'generator_loss': loss,
                    'discriminator_loss': discriminator_loss,
                    'val_loss': mean_val_loss,
                    'beta': beta
                    }, gen_name)        
        
        # Compute alpha e beta
        mean_gen_loss = total_gen_loss / num_samples_seen
        mean_dis_loss = total_dis_loss / num_samples_seen

        gen_order = torch.floor(torch.log10(mean_gen_loss))
        dis_order = 0 if mean_dis_loss == 0 else torch.floor(torch.log10(mean_dis_loss))
        b_pow = gen_order - dis_order 
        if b_pow > 0:
            b_pow = b_pow
        beta = pow(10.0, b_pow)
        
        
        #Ad ogni epoca stampiamo la loss
        file = open(filename, 'a') # w+ is to append new lines to the file
        file.write(
        'EPOCH ' + str(epoch+1) +
        '\n\t -> Discriminative Loss during D Training = ' + str(mean_dis_loss.item()) + ', during G Training = ' + str(discriminator_loss_gen.item()) +
        '\n\t -> Generative Loss = ' + str(loss.item()) + ' ---> alpha * ' + str(mean_gen_loss.item()) + ' beta * ' + str(mean_dis_loss.item()) + 
        '\n\t -> Validation Loss = ' + str(mean_val_loss.item()) + '\n\n')
        file.flush()
        file.close()     
        
        print('EPOCH ' + str(epoch+1) +
        '\n\t -> Discriminative Loss during D Training = ' + str(mean_dis_loss.item()) + ', during G Training = ' + str(discriminator_loss_gen.item()) +
        '\n\t -> Generative Loss = ' + str(loss.item()) + ' ---> alpha * ' + str(mean_gen_loss.item()) + ' beta * ' + str(mean_dis_loss.item()) + 
        '\n\t -> Validation Loss = ' + str(mean_val_loss.item()) + '\n\n')
        
        dis_name = os.path.join(TRAIN_CHECKPOINTS_DIR_PATH, 'DIS_'+name_to_save)
        torch.save({
            'epoch': epoch+start,
            'model_state_dict': discriminator.state_dict(),
            'optimizer_state_dict': optimizer_dis.state_dict(),
            'generator_loss': loss,
            'discriminator_loss': discriminator_loss,
            'val_loss': 0,
            'beta': beta
            }, dis_name)
    
        gen_name = os.path.join(TRAIN_CHECKPOINTS_DIR_PATH, 'GEN_'+name_to_save)
        torch.save({
            'epoch': epoch+start,
            'model_state_dict': generator.state_dict(),
            'optimizer_state_dict': optimizer_gen.state_dict(),
            'generator_loss': loss,
            'discriminator_loss':discriminator_loss,
            'val_loss': mean_val_loss,
            'beta': beta
            }, gen_name)


In [15]:
name_to_save = 'reshapeAfterVit_V1_100'

In [16]:
#train_with_val(trainloader, validloader, generator, discriminator, optimizer_gen, optimizer_dis, loss_gen, loss_dis, name_to_save=name_to_save, epoches=20, start=0, beta=1.0, best_val=float('inf'), device=device)

### Restore and Resume training

In [ ]:
TRAIN_CHECKPOINTS_DIR_PATH = '.'
name_to_save = 'reshapeAfterVit_V1'

# Restore discriminator
dis_name = os.path.join(TRAIN_CHECKPOINTS_DIR_PATH, 'DIS_'+name_to_save)
dis_checkpoint = torch.load(dis_name)
discriminator.load_state_dict(dis_checkpoint['model_state_dict'])
optimizer_dis.load_state_dict(dis_checkpoint['optimizer_state_dict'])
discriminator_loss = dis_checkpoint['discriminator_loss']
epoch = dis_checkpoint['epoch']

#best val loss
ckp = torch.load(os.path.join(TRAIN_CHECKPOINTS_DIR_PATH, 'GEN_BestVal_'+name_to_save))
best_val_loss = ckp['val_loss']

# Restore generator
gen_name = os.path.join(TRAIN_CHECKPOINTS_DIR_PATH, 'GEN_'+name_to_save)
gen_checkpoint = torch.load(gen_name)
generator.load_state_dict(gen_checkpoint['model_state_dict'])
optimizer_gen.load_state_dict(gen_checkpoint['optimizer_state_dict'])
discriminator_loss = gen_checkpoint['discriminator_loss']
epoch = gen_checkpoint['epoch']
mean_val_loss = gen_checkpoint['val_loss']
beta = gen_checkpoint['beta']

train_with_val(trainloader, validloader, generator, discriminator, optimizer_gen, optimizer_dis, loss_gen, loss_dis, name_to_save=name_to_save, epoches=200, start=epoch, beta=beta, best_val=best_val_loss, device=device)

Epoch :   0%|          | 0/200 [00:00<?, ?it/s]

Song n.0 on 2664
Song n.1 on 2664
Song n.2 on 2664
Song n.3 on 2664
Song n.4 on 2664
Song n.5 on 2664
Song n.6 on 2664
Song n.7 on 2664
Song n.8 on 2664
Song n.9 on 2664
Song n.10 on 2664
Song n.11 on 2664
Song n.12 on 2664
Song n.13 on 2664
Song n.14 on 2664
Song n.15 on 2664
Song n.16 on 2664
Song n.17 on 2664
Song n.18 on 2664
Song n.19 on 2664
Song n.20 on 2664
Song n.21 on 2664
Song n.22 on 2664
Song n.23 on 2664
Song n.24 on 2664
Song n.25 on 2664
Song n.26 on 2664
Song n.27 on 2664
Song n.28 on 2664
Song n.29 on 2664
Song n.30 on 2664
Song n.31 on 2664
Song n.32 on 2664
Song n.33 on 2664
Song n.34 on 2664
Song n.35 on 2664
Song n.36 on 2664
Song n.37 on 2664
Song n.38 on 2664
Song n.39 on 2664
Song n.40 on 2664
Song n.41 on 2664
Song n.42 on 2664
Song n.43 on 2664
Song n.44 on 2664
Song n.45 on 2664
Song n.46 on 2664
Song n.47 on 2664
Song n.48 on 2664
Song n.49 on 2664
Song n.50 on 2664
Song n.51 on 2664
Song n.52 on 2664
Song n.53 on 2664
Song n.54 on 2664
Song n.55 on 2664
So

Song n.437 on 2664
Song n.438 on 2664
Song n.439 on 2664
Song n.440 on 2664
Song n.441 on 2664
Song n.442 on 2664
Song n.443 on 2664
Song n.444 on 2664
Song n.445 on 2664
Song n.446 on 2664
Song n.447 on 2664
Song n.448 on 2664
Song n.449 on 2664
Song n.450 on 2664
Song n.451 on 2664
Song n.452 on 2664
Song n.453 on 2664
Song n.454 on 2664
Song n.455 on 2664
Song n.456 on 2664
Song n.457 on 2664
Song n.458 on 2664
Song n.459 on 2664
Song n.460 on 2664
Song n.461 on 2664
Song n.462 on 2664
Song n.463 on 2664
Song n.464 on 2664
Song n.465 on 2664
Song n.466 on 2664
Song n.467 on 2664
Song n.468 on 2664
Song n.469 on 2664
Song n.470 on 2664
Song n.471 on 2664
Song n.472 on 2664
Song n.473 on 2664
Song n.474 on 2664
Song n.475 on 2664
Song n.476 on 2664
Song n.477 on 2664
Song n.478 on 2664
Song n.479 on 2664
Song n.480 on 2664
Song n.481 on 2664
Song n.482 on 2664
Song n.483 on 2664
Song n.484 on 2664
Song n.485 on 2664
Song n.486 on 2664
Song n.487 on 2664
Song n.488 on 2664
Song n.489 o

Song n.869 on 2664
Song n.870 on 2664
Song n.871 on 2664
Song n.872 on 2664
Song n.873 on 2664
Song n.874 on 2664
Song n.875 on 2664
Song n.876 on 2664
Song n.877 on 2664
Song n.878 on 2664
Song n.879 on 2664
Song n.880 on 2664
Song n.881 on 2664
Song n.882 on 2664
Song n.883 on 2664
Song n.884 on 2664
Song n.885 on 2664
Song n.886 on 2664
Song n.887 on 2664
Song n.888 on 2664
Song n.889 on 2664
Song n.890 on 2664
Song n.891 on 2664
Song n.892 on 2664
Song n.893 on 2664
Song n.894 on 2664
Song n.895 on 2664
Song n.896 on 2664
Song n.897 on 2664
Song n.898 on 2664
Song n.899 on 2664
Song n.900 on 2664
Song n.901 on 2664
Song n.902 on 2664
Song n.903 on 2664
Song n.904 on 2664
Song n.905 on 2664
Song n.906 on 2664
Song n.907 on 2664
Song n.908 on 2664
Song n.909 on 2664
Song n.910 on 2664
Song n.911 on 2664
Song n.912 on 2664
Song n.913 on 2664
Song n.914 on 2664
Song n.915 on 2664
Song n.916 on 2664
Song n.917 on 2664
Song n.918 on 2664
Song n.919 on 2664
Song n.920 on 2664
Song n.921 o

Song n.1286 on 2664
Song n.1287 on 2664
Song n.1288 on 2664
Song n.1289 on 2664
Song n.1290 on 2664
Song n.1291 on 2664
Song n.1292 on 2664
Song n.1293 on 2664
Song n.1294 on 2664
Song n.1295 on 2664
Song n.1296 on 2664
Song n.1297 on 2664
Song n.1298 on 2664
Song n.1299 on 2664
Song n.1300 on 2664
Song n.1301 on 2664
Song n.1302 on 2664
Song n.1303 on 2664
Song n.1304 on 2664
Song n.1305 on 2664
Song n.1306 on 2664
Song n.1307 on 2664
Song n.1308 on 2664
Song n.1309 on 2664
Song n.1310 on 2664
Song n.1311 on 2664
Song n.1312 on 2664
Song n.1313 on 2664
Song n.1314 on 2664
Song n.1315 on 2664
Song n.1316 on 2664
Song n.1317 on 2664
Song n.1318 on 2664
Song n.1319 on 2664
Song n.1320 on 2664
Song n.1321 on 2664
Song n.1322 on 2664
Song n.1323 on 2664
Song n.1324 on 2664
Song n.1325 on 2664
Song n.1326 on 2664
Song n.1327 on 2664
Song n.1328 on 2664
Song n.1329 on 2664
Song n.1330 on 2664
Song n.1331 on 2664
Song n.1332 on 2664
Song n.1333 on 2664
Song n.1334 on 2664
Song n.1335 on 2664


...

Song n.1696 on 2664
Song n.1697 on 2664
Song n.1698 on 2664
Song n.1699 on 2664
Song n.1700 on 2664
Song n.1701 on 2664
Song n.1702 on 2664
Song n.1703 on 2664
Song n.1704 on 2664
Song n.1705 on 2664
Song n.1706 on 2664
Song n.1707 on 2664
Song n.1708 on 2664
Song n.1709 on 2664
Song n.1710 on 2664
Song n.1711 on 2664
Song n.1712 on 2664
Song n.1713 on 2664
Song n.1714 on 2664
Song n.1715 on 2664
Song n.1716 on 2664
Song n.1717 on 2664
Song n.1718 on 2664
Song n.1719 on 2664
Song n.1720 on 2664
Song n.1721 on 2664
Song n.1722 on 2664
Song n.1723 on 2664
Song n.1724 on 2664
Song n.1725 on 2664
Song n.1726 on 2664
Song n.1727 on 2664
Song n.1728 on 2664
Song n.1729 on 2664
Song n.1730 on 2664
Song n.1731 on 2664
Song n.1732 on 2664
Song n.1733 on 2664
Song n.1734 on 2664
Song n.1735 on 2664
Song n.1736 on 2664
Song n.1737 on 2664
Song n.1738 on 2664
Song n.1739 on 2664
Song n.1740 on 2664
Song n.1741 on 2664
Song n.1742 on 2664
Song n.1743 on 2664
Song n.1744 on 2664
Song n.1745 on 2664


Song n.2106 on 2664
Song n.2107 on 2664
Song n.2108 on 2664
Song n.2109 on 2664
Song n.2110 on 2664
Song n.2111 on 2664
Song n.2112 on 2664
Song n.2113 on 2664
Song n.2114 on 2664
Song n.2115 on 2664
Song n.2116 on 2664
Song n.2117 on 2664
Song n.2118 on 2664
Song n.2119 on 2664
Song n.2120 on 2664
Song n.2121 on 2664
Song n.2122 on 2664
Song n.2123 on 2664
Song n.2124 on 2664
Song n.2125 on 2664
Song n.2126 on 2664
Song n.2127 on 2664
Song n.2128 on 2664
Song n.2129 on 2664
Song n.2130 on 2664
Song n.2131 on 2664
Song n.2132 on 2664
Song n.2133 on 2664
Song n.2134 on 2664
Song n.2135 on 2664
Song n.2136 on 2664
Song n.2137 on 2664
Song n.2138 on 2664
Song n.2139 on 2664
Song n.2140 on 2664
Song n.2141 on 2664
Song n.2142 on 2664
Song n.2143 on 2664
Song n.2144 on 2664
Song n.2145 on 2664
Song n.2146 on 2664
Song n.2147 on 2664
Song n.2148 on 2664
Song n.2149 on 2664
Song n.2150 on 2664
Song n.2151 on 2664
Song n.2152 on 2664
Song n.2153 on 2664
Song n.2154 on 2664
Song n.2155 on 2664


Song n.2516 on 2664
Song n.2517 on 2664
Song n.2518 on 2664
Song n.2519 on 2664
Song n.2520 on 2664
Song n.2521 on 2664
Song n.2522 on 2664
Song n.2523 on 2664
Song n.2524 on 2664
Song n.2525 on 2664
Song n.2526 on 2664
Song n.2527 on 2664
Song n.2528 on 2664
Song n.2529 on 2664
Song n.2530 on 2664
Song n.2531 on 2664
Song n.2532 on 2664
Song n.2533 on 2664
Song n.2534 on 2664
Song n.2535 on 2664
Song n.2536 on 2664
Song n.2537 on 2664
Song n.2538 on 2664
Song n.2539 on 2664
Song n.2540 on 2664
Song n.2541 on 2664
Song n.2542 on 2664
Song n.2543 on 2664
Song n.2544 on 2664
Song n.2545 on 2664
Song n.2546 on 2664
Song n.2547 on 2664
Song n.2548 on 2664
Song n.2549 on 2664
Song n.2550 on 2664
Song n.2551 on 2664
Song n.2552 on 2664
Song n.2553 on 2664
Song n.2554 on 2664
Song n.2555 on 2664
Song n.2556 on 2664
Song n.2557 on 2664
Song n.2558 on 2664
Song n.2559 on 2664
Song n.2560 on 2664
Song n.2561 on 2664
Song n.2562 on 2664
Song n.2563 on 2664
Song n.2564 on 2664
Song n.2565 on 2664


Epoch :   4%|▍         | 8/200 [3:30:31<84:02:48, 1575.88s/it]

Song n.0 on 2664
Song n.1 on 2664
Song n.2 on 2664
Song n.3 on 2664
Song n.4 on 2664
Song n.5 on 2664
Song n.6 on 2664
Song n.7 on 2664
Song n.8 on 2664
Song n.9 on 2664
Song n.10 on 2664
Song n.11 on 2664
Song n.12 on 2664
Song n.13 on 2664
Song n.14 on 2664
Song n.15 on 2664
Song n.16 on 2664
Song n.17 on 2664
Song n.18 on 2664
Song n.19 on 2664
Song n.20 on 2664
Song n.21 on 2664
Song n.22 on 2664
Song n.23 on 2664
Song n.24 on 2664
Song n.25 on 2664
Song n.26 on 2664
Song n.27 on 2664
Song n.28 on 2664
Song n.29 on 2664
Song n.30 on 2664
Song n.31 on 2664
Song n.32 on 2664
Song n.33 on 2664
Song n.34 on 2664
Song n.35 on 2664
Song n.36 on 2664
Song n.37 on 2664
Song n.38 on 2664
Song n.39 on 2664
Song n.40 on 2664
Song n.41 on 2664
Song n.42 on 2664
Song n.43 on 2664
Song n.44 on 2664
Song n.45 on 2664
Song n.46 on 2664
Song n.47 on 2664
Song n.48 on 2664
Song n.49 on 2664
Song n.50 on 2664
Song n.51 on 2664
Song n.52 on 2664
Song n.53 on 2664
Song n.54 on 2664
Song n.55 on 2664
So